# StyleGan2-ADA with transfer learning


## Installing StyleGAN2-ADA on Google Drive

Using TF1 as StyleGAN2-ADA only works with TF1.


In [3]:
%tensorflow_version 1.x
!nvidia-smi

Fri Nov  4 05:27:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    26W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Installing numpy 1.19.5

In [ ]:
!pip install numpy==1.19.5
import numpy
print(numpy.__version__)

1.21.6


Mounting Drive to the Colab notebook: 

In [ ]:
from google.colab import drive
from pathlib import Path

content_path = Path('/').absolute() / 'content'
drive_path = content_path / 'drive'
drive.mount(str(drive_path))

Mounted at /content/drive


Install StyleGAN2-ADA on Drive.Everything will be available on your Google Drive in the folder **StyleGAN2-ADA**

In [ ]:
stylegan2_repo_url  = 'https://github.com/NVlabs/stylegan2-ada' 
project_path = drive_path / 'MyDrive' / 'StyleGAN2-ADA'
stylegan2_repo_path = project_path / 'stylegan2-ada'

Create project folder if inexistant
if not project_path.is_dir():
    %mkdir "{project_path}"
%cd "{project_path}"

for dir in ['in', 'out', 'datasets', 'training']:
    if not (project_path / dir).is_dir():
        %mkdir {dir}
if not (project_path / 'datasets' / 'source').is_dir():
    %mkdir "{project_path / 'datasets' / 'source'}"

# Download StyleGAN2-ada
!git config --global user.name "" #Your Github username
!git config --global user.email "" #Your Github email
if stylegan2_repo_path.is_dir():
    !git -C "{stylegan2_repo_path}" fetch origin
    !git -C "{stylegan2_repo_path}" checkout origin/main -- *.py
else:
    print("Install StyleGAN2-ADA")
    !git clone {stylegan2_repo_url}

## Training

Once you have installed StyleGAN2-ADA on your Google Drive and set up the working directory, you can upload your training dataset images in the associated folder.

In [ ]:
dataset_name = 'brain_tumor'
datasets_source_path = project_path / 'datasets' / 'source' / (dataset_name + '.zip')
if datasets_source_path.is_file():
    print("Dataset ready for import.")
else:
    print('Upload your images dataset as {}'.format(datasets_source_path))

Dataset ready for import.


In [ ]:
#!rm -rf /content/dataset

Download zipped dataset on Drive and extract it in the local session.

In [ ]:
local_dataset_path = content_path / 'dataset'
if not local_dataset_path.is_dir():
    print("Importing dataset...")
    %mkdir "{local_dataset_path}"
    %cp -a "{project_path / 'datasets' / 'source' / (dataset_name + '.zip')}" "{local_dataset_path}"
    print("Zip file succesfuly imported")
else:
    print('Zip file allready imported')

import zipfile
with zipfile.ZipFile(str(local_dataset_path / (dataset_name + '.zip')), 'r') as zip_ref:
    zip_ref.extractall(str(local_dataset_path))
print('Extraction completed')

Importing dataset...
Zip file succesfuly imported
Extraction completed


### Convert dataset to .tfrecords

Convert our image dataset to a format that StyleGAN2-ADA can read:`.tfrecords`.


In [ ]:
local_images_path = local_dataset_path / 'brain_tumor'
local_dataset_path /= 'tfr'

if (local_dataset_path).is_dir():
    print('\N{Heavy Exclamation Mark Symbol} Dataset already created \N{Heavy Exclamation Mark Symbol}')
    print('Delete current dataset folder ({}) to regenerate tfrecords.'.format(local_dataset_path))
else:
    %mkdir "{local_dataset_path}"
    !python "{stylegan2_repo_path / 'dataset_tool.py'}" create_from_images \
        "{local_dataset_path}" "{local_images_path}"

Loading images from "/content/dataset/brain_tumor"
Creating dataset "/content/dataset/tfr"
/content/drive/MyDrive/StyleGAN2-ADA/stylegan2-ada/dataset_tool.py:96: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[quant.tostring()]))}))
Added 154 images.


In [ ]:
import numpy
print(numpy.__version__)
print(local_dataset_path)

1.19.5
/content/dataset/tfr


In [ ]:
!python "{stylegan2_repo_path / 'train.py'}" --help

usage: train.py [-h] --outdir DIR [--gpus INT] [--snap INT] [--seed INT] [-n]
                --data PATH [--res INT] [--mirror BOOL] [--metrics LIST]
                [--metricdata PATH]
                [--cfg {auto,stylegan2,paper256,paper512,paper1024,cifar,cifarbaseline}]
                [--gamma FLOAT] [--kimg INT] [--aug {noaug,ada,fixed,adarv}]
                [--p FLOAT] [--target TARGET]
                [--augpipe {blit,geom,color,filter,noise,cutout,bg,bgc,bgcf,bgcfn,bgcfnc}]
                [--cmethod {nocmethod,bcr,zcr,pagan,wgangp,auxrot,spectralnorm,shallowmap,adropout}]
                [--dcap FLOAT] [--resume RESUME] [--freezed INT]

Train a GAN using the techniques described in the paper
"Training Generative Adversarial Networks with Limited Data".

optional arguments:
  -h, --help            show this help message and exit

general options:
  --outdir DIR          Where to save the results (required)
  --gpus INT            Number of GPUs to use (default: 1 gpu)
  --sn

In [ ]:
print(project_path)

/content/drive/MyDrive/StyleGAN2-ADA


In [ ]:
training_path = project_path / 'training' / 'brain_tumor_afhqwild_pretrained'
if not training_path.is_dir():
    %mkdir "{training_path}"

#how often should the model generate samples and a .pkl file
snapshot_count = 10
#should the images be mirrored
mirrored = True
metric_list = "fid50k_full,kid50k_full"
#augments
augs = 'bgc'
kimg = 1000

resume_from = project_path / 'training' / 'brain_tumor_afhqwild_pretrained'/ '00000-tfr-mirror-auto1-kimg1000-bgc-resumecustom' / 'network-snapshot-000160.pkl'#project_path / 'pretrained_models' / 'afhqwild.pkl'

!python "{stylegan2_repo_path / 'train.py'}" --outdir="{training_path}" \
    --data="{local_dataset_path}" --resume="{resume_from}" \
    --snap={snapshot_count} --augpipe={augs} \
    --mirror={mirrored} --metrics={metric_list} --kimg={kimg}

tcmalloc: large alloc 4294967296 bytes == 0x8248000 @  0x7f3fa7797001 0x7f3fa49db54f 0x7f3fa4a2bb58 0x7f3fa4a2fb17 0x7f3fa4ace203 0x593835 0x548c51 0x5127f1 0x549e0e 0x4bca8a 0x532b86 0x594a96 0x548cc1 0x5127f1 0x549576 0x4bca8a 0x5134a6 0x549576 0x4bca8a 0x5134a6 0x549e0e 0x4bca8a 0x5134a6 0x593dd7 0x5118f8 0x549576 0x604173 0x5f5506 0x5f8c6c 0x5f9206 0x64faf2
tcmalloc: large alloc 4294967296 bytes == 0x7f3dbe828000 @  0x7f3fa77951e7 0x7f3fa49db46e 0x7f3fa4a2bc7b 0x7f3fa4a2c35f 0x7f3fa4ace103 0x593835 0x548c51 0x5127f1 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x548ae9 0x5127f1 0x593dd7 0x5118f8 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x549e0e
tcmalloc: large alloc 4294967296 bytes == 0x7f3cbd826000 @  0x7f3fa77951e7 0x7f3fa49db46e 0x7f3fa4a2bc7b 0x7f3fa4a2c35f 0x7f3f505b5235 0x7f3f4ff38792 0x7f3f4ff38d42 0x7f3f4fef1aee 0x59371f 0x548c51 0x51566f 0x593dd7 0x511e2c 0x549e0e 0x4bcb19 0x5134a6 0x549576 0x593f

In [ ]:
print(project_path)

/content/drive/MyDrive/StyleGAN2-ADA


Generate images using a pre-trained network once everything is set up.

In [ ]:
%pip install opensimplex
!python "{stylegan2_repo_path / 'generate.py'}" generate-images --help 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
usage: generate.py [-h] --network NETWORK_PKL
                   (--seeds SEEDS | --dlatents DLATENTS_NPZ)
                   [--trunc TRUNCATION_PSI] [--class CLASS_IDX] --outdir DIR

Generate images using pretrained network pickle.

optional arguments:
  -h, --help            show this help message and exit
  --network NETWORK_PKL
                        Network pickle filename
  --seeds SEEDS         List of random seeds
  --dlatents DLATENTS_NPZ
                        Generate images for saved dlatents
  --trunc TRUNCATION_PSI
                        Truncation psi (default: 0.5)
  --class CLASS_IDX     Class label (default: unconditional)
  --outdir DIR          Where to save the output images

examples:

  # Generate curated MetFaces images without truncation (Fig.10 left)
  python generate.py --outdir=out --trunc=1 --seeds=85,265,297,849 \
      --network=https://nvlabs-fi-cdn.nvi

In [ ]:
print(stylegan2_repo_path)

/content/drive/MyDrive/StyleGAN2-ADA/stylegan2-ada


In [ ]:
from numpy import random
seed_init = random.randint(10000)
nbr_images = 500

generation_from = '/content/drive/MyDrive/StyleGAN2-ADA/training/brain_tumor_afhqcat_pretrained/00001-tfr-mirror-auto1-kimg1000-bgc-resumecustom/network-snapshot-000160.pkl'

!python "{stylegan2_repo_path / 'generate.py'}" --outdir="{project_path / 'out' / '500_images_afhqcat'}" --trunc=0.7 --seeds={seed_init}-{seed_init+nbr_images-1}  --network={generation_from}

Loading networks from "/content/drive/MyDrive/StyleGAN2-ADA/training/brain_tumor_afhqcat_pretrained/00001-tfr-mirror-auto1-kimg1000-bgc-resumecustom/network-snapshot-000160.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Generating image for seed 6265 (0/500) ...
Generating image for seed 6266 (1/500) ...
Generating image for seed 6267 (2/500) ...
Generating image for seed 6268 (3/500) ...
Generating image for seed 6269 (4/500) ...
Generating image for seed 6270 (5/500) ...
Generating image for seed 6271 (6/500) ...
Generating image for seed 6272 (7/500) ...
Generating image for seed 6273 (8/500) ...
Generating image for seed 6274 (9/500) ...
Generating image for seed 6275 (10/500) ...
Generating image for seed 6276 (11/500) ...
Generating image for seed 6277 (12/500) ...
Generating image for seed 6278 (13/500) ...
Generating image for seed 6279 (14/500) ...
Generating image for seed 6280 (15/500)